In [1]:
#Para correr las funciones de la librería libmf de este notebook, debes clonar 
#el repo de https://github.com/PorkShoulderHolder/python-libmf.git
#subir este notebook dentro de la carpeta /libmf/tests/

In [ ]:
#si queres correr este notebook a través de docker con la imagen de
#jupyer numerical, debes instalar los siguientes paquetes:
#pip install libmf
#pip install colorama

In [2]:
import sys
import numpy as np
import types
import scipy
from scipy.sparse import random
from scipy import stats
import pandas as pd
import copy
import random
from functools import partial
from colorama import init

import libmf
from libmf import mf

Using file found in /home/miuser/.local/lib/python3.6/site-packages:
/home/miuser/.local/lib/python3.6/site-packages/libmf.cpython-36m-x86_64-linux-gnu.so


In [3]:
def constante(x,useless):
    return x

In [4]:
def perform(fun, *args):
    return fun(*args)

In [5]:
def nan_or_value(x, f, *args):
    if(x==1):
        return perform(f,*args)
    else:
        return np.nan

In [6]:
nan_or_value_vec=np.vectorize(nan_or_value, otypes=[object])

In [7]:
def crear_matriz_rala(n_ren,m_col,prop_elem_difer_nan,func_rand,*args_func_rand):
    tot=int(n_ren*m_col)
    k=int(tot*prop_elem_difer_nan)
    M=np.concatenate((np.zeros(tot-k).astype(int), np.ones(k).astype(int)))
    M=np.array(random.sample(list(M),tot))
    M=M.reshape(n_ren,m_col)
    M=nan_or_value_vec(M,func_rand,*args_func_rand).astype(float)
    return M

In [8]:
def crear_matriz_rala_compacta(n_ren, m_col, prop_elem_difer_nan):
    tot=int(n_ren*m_col)
    k=int(tot*prop_elem_difer_nan)
    M=random.sample(range(tot), k)
    columnas=np.mod(M,m_col)
    renglones=np.divide(M,m_col).astype(int)
    aleatorios=np.random.randint(1, 6, k)
    M=np.concatenate([renglones, columnas, aleatorios])
    M=M.reshape(3,k).T
    return M

In [9]:
def compactar_matriz_rala(M):
    if type(M) is not np.ndarray:
        sys.exit('M debe ser de tipo numpy.ndarray')
    elif np.isnan(M).sum()==len(M):
        sys.exit('Todos los elementos de M son nan')
    MC=np.zeros((M[np.isnan(M)==False].size,3))
    k = 0
    n,m=M.shape
    for i in range(n): 
        for j in range(m): 
            if (~np.isnan(M[i,j])): 
                MC[k,0] = i
                MC[k,1] = j
                MC[k,2] = M[i,j] 
                k += 1
    return MC

In [10]:
#Elegiremos un formato de general para visualizar las matrices exhibidas como ejemplo
np.set_printoptions(sign=' ', precision=3)
#np.set_printoptions(sign=' ',formatter={'float': lambda x: "{0:0.3f}".format(x)})

In [11]:
def modelo_base_ref(MC):
    df=pd.DataFrame(MC,columns=['row','col','score'])
    #Media total
    c=df['score'].mean(axis=0)

    #media por columna
    mean_by_col=df[['col','score']].groupby(['col']).mean()
    mean_by_col.rename(columns={'score':'mean_score_by_col'}, inplace=True)
    #media por renglon
    mean_by_row=df[['row','score']].groupby(['row']).mean()
    mean_by_row.rename(columns={'score':'mean_score_by_row'}, inplace=True)

    #se unen todas los resultados obtenidos, incluyendo la columna de
    #índice de columnas y renglones
    df=df.set_index('col').join(mean_by_col,how='left')
    df['col']=df.index
    df=df.set_index('row').join(mean_by_row,how='left')
    df['row']=df.index
    df=df[['row','col','mean_score_by_row','mean_score_by_col']]
    df['total_score_mean']=c
    
    return df

In [ ]:
#Esta función filtra los 'indices de renglones (usuarios)' o
#'indices de columna (peliculas)' cuyo conteo total es mayor a n
def filtrar(MC,ren_o_col,n):
    MC_df=pd.DataFrame(MC,columns=['row','col','score'])
    
    #este pequeño bloque calcula el total de elementos por columna o renglón
    count_by=MC_df[[ren_o_col,'score']].groupby([ren_o_col]).count()
    count_by.rename(columns={'score':'tot_counted'}, inplace=True)
    count_by[ren_o_col]=count_by.index
    count_by=count_by[[ren_o_col,'tot_counted']]
    
    #en este bloque se filtran los elementos de la matriz cuyo conteo por
    #columna o renglón tienen más de n elementos
    count_by=count_by[count_by.apply(lambda x: x['tot_counted'] > n, axis=1)]
    count_by=MC_df.set_index(ren_o_col).join(count_by.set_index(ren_o_col),how='inner')
    count_by[ren_o_col]=count_by.index
    count_by=count_by[['row','col','score']]
    
    return count_by.to_numpy()

In [12]:
#Crear matriz rala de tamaño 20x5, donde el 25% de sus elementos
#será distinto de nan, señalando la distribución de probabilidad
#que tendrán dichos elementos
M=crear_matriz_rala(20,5,0.25,stats.randint.rvs,*[1,6])
#Puedes cambiar la distribución aleatoria, prueba con las
#distribuciones existentes en scipy.stats o la función constante
#R=crear_matriz_rala(20,5,0.5,scipy.stats.uniform.rvs,*[1,4])
#R=crear_matriz_rala(20,5,0.2,stats.binom.rvs,*[5,0.2])
#R=crear_matriz_rala(20,5,0.5,constante,*[1,0])
print('Matriz M:\n',M)
print('Tamaño M:\n',M.shape)
print('Numero de elementos diferentes de nan:\n',M[np.isnan(M)==False].size)

Matriz M:
 [[ nan  nan  nan  nan  nan]
 [  3.   3.  nan   4.  nan]
 [  2.  nan  nan  nan   5.]
 [ nan  nan   5.   5.  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan   5.   4.]
 [ nan  nan  nan  nan  nan]
 [ nan  nan   4.  nan  nan]
 [ nan  nan  nan  nan   4.]
 [  3.  nan  nan   3.  nan]
 [ nan  nan   2.  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan   2.   4.  nan  nan]
 [ nan  nan  nan  nan   4.]
 [ nan  nan   4.  nan  nan]
 [  4.  nan  nan   4.  nan]
 [  2.  nan  nan  nan   4.]
 [ nan  nan  nan   5.   4.]
 [ nan  nan  nan  nan   1.]
 [ nan  nan  nan  nan  nan]]
Tamaño M:
 (20, 5)
Numero de elementos diferentes de nan:
 25


In [13]:
#Expresión compacta de la matriz rala (con 3 columnas: col1,col2,col3),
#que señala el índice de renglón (col1) y columna (col2) de la matriz R,
#correspondiente a los elementos distitnos de nan, así como el valor
#de dicho elemento (col3)
MC=compactar_matriz_rala(M)
print('Expresión compacta de M (MC):\n',MC)
print('Tamaño MC:\n',MC.shape)

Expresión compacta de M (MC):
 [[  1.   0.   3.]
 [  1.   1.   3.]
 [  1.   3.   4.]
 [  2.   0.   2.]
 [  2.   4.   5.]
 [  3.   2.   5.]
 [  3.   3.   5.]
 [  5.   3.   5.]
 [  5.   4.   4.]
 [  7.   2.   4.]
 [  8.   4.   4.]
 [  9.   0.   3.]
 [  9.   3.   3.]
 [ 10.   2.   2.]
 [ 12.   1.   2.]
 [ 12.   2.   4.]
 [ 13.   4.   4.]
 [ 14.   2.   4.]
 [ 15.   0.   4.]
 [ 15.   3.   4.]
 [ 16.   0.   2.]
 [ 16.   4.   4.]
 [ 17.   3.   5.]
 [ 17.   4.   4.]
 [ 18.   4.   1.]]
Tamaño MC:
 (25, 3)


In [14]:
#Entre las funciones de la librería libmf programado en python
#se tiene un para generar matrices ralas compactas sin embargo
#se observa que puede arrojar más de un valor para un mismo
#elemento de la matriz rala, vease abajo el renglón [0,1]
#tiene 3 valores asignados (0.166,0.217,0.556).

#El 1er parámetro señala indica que se seleccionará un valor
#de entre 0 y 19 para la 1er columna (que representa el índice
#de los renglones de la matriz rala que expresa); el 2do
#parámetro indica que se seleccionara al azar valores entre
#0 y 4 (que representa el índice de las columnas de la matriz
#rala que expresa); el 3er parámetro indica el número de elementos
#a generar

#Fijaremos una semilla para reproducibilidad
np.random.seed(102)
MC=mf.generate_test_data(20, 5, 25)

print('Matriz generada con mf.generate_test_data(20,5,25):\n')
#np.set_printoptions(sign=' ',formatter={'float': lambda x: "{0:0.3f}".format(x)})
for i in range(MC.shape[0]):
    if MC[i,0]==0:
        print('\033[1;30;41m',MC[i,:])
    else:
        print('\033[0;0;0m',MC[i,:])
#print('Matriz generada con mf.generate_test_data(20,5,25):\n',M_compacta)
print('Tamaño MC:\n',MC.shape)


Matriz generada con mf.generate_test_data(20,5,25):

 [ 0.     1.     0.166]
 [ 19.      4.      0.306]
 [ 14.      3.      0.147]
 [ 18.      4.      0.223]
 [ 18.     1.     0.54]
 [ 15.      3.      0.749]
 [ 2.     2.     0.428]
 [ 11.      3.      0.105]
 [ 9.     3.     0.753]
 [ 8.     5.     0.598]
 [ 13.      0.      0.487]
 [ 8.     4.     0.975]
 [ 9.     2.     0.802]
 [ 15.      3.      0.333]
 [ 7.     5.     0.549]
 [ 4.     5.     0.312]
 [ 16.      4.      0.062]
 [ 6.     1.     0.903]
 [ 18.      0.      0.861]
 [ 3.    3.    0.63]
 [ 0.     1.     0.217]
 [ 0.     1.     0.556]
 [ 16.      5.      0.109]
 [ 6.     5.     0.551]
 [ 7.     0.     0.581]
Tamaño MC:
 (25, 3)


In [76]:
#El presente trabajo plantea trabajar con bases de películas, donde
#los ínidices i de renglones representan usuarios, los ínidces j de columnas
#las películas, y el valor de elemento de la matriz la calificación
#asignada a la película j por el usuari i
#Por lo anterior, para crear matrices compactas ralas usaremos funciones propias
#donde un usuario no califique más de una vez una misma película (lo cual 
#sucediería si se genera la matriz de calificaciones con la función de la
#paquetería libmf). La calificación será un no. aleatorio entre 1 y 5
#generada con una dist. uniforme discreta
MC=crear_matriz_rala_compacta(20,5,0.25)
print('Expresión compacta de M (MC) ordenada por renglon:\n',MC[np.argsort(MC[:,0])])
print('Tamaño MC:\n',MC.shape)

Expresión compacta de M (MC) ordenada por renglon:
 [[ 0  3  4]
 [ 0  2  5]
 [ 1  4  3]
 [ 1  2  2]
 [ 2  3  5]
 [ 2  2  4]
 [ 4  3  3]
 [ 4  2  3]
 [ 4  0  1]
 [ 6  2  5]
 [ 8  3  3]
 [ 9  2  1]
 [10  1  3]
 [10  3  2]
 [12  1  4]
 [12  4  3]
 [13  2  3]
 [15  1  1]
 [15  2  3]
 [16  2  4]
 [16  4  5]
 [16  1  2]
 [16  3  1]
 [17  3  3]
 [18  4  2]]
Tamaño MC:
 (25, 3)


In [77]:
#Aquí una forma de gardar la matriz compacta en un txt
#que podría ser utilizado directamente con las funciones
#de libmf a través de comandos bash
np.savetxt('MC.txt', MC, delimiter=' ', fmt="%.0f")
f = open('MC.txt', 'r')
file_contents = f.read()
print (file_contents)
f.close()

16 3 1
16 1 2
12 1 4
10 1 3
15 2 3
15 1 1
0 2 5
1 2 2
6 2 5
4 2 3
18 4 2
13 2 3
2 3 5
16 2 4
2 2 4
17 3 3
4 3 3
16 4 5
10 3 2
1 4 3
8 3 3
12 4 3
9 2 1
0 3 4
4 0 1



In [78]:
#Para plantear un modelo base de referencia, se calculan los medias
#por renglones (mean_ren) y por columnas (mean_col), así como 
#una media total (mean_tot), luego se sustituyen los valores de 
#la matriz (x_ij) por: mean_ren + mean_col - mean_tot
#La función modelo_base_ref regresa los promedios por renglon,
#columna y total. Con ello se pude construir el modelo base referencia
promedios=modelo_base_ref(MC)
display(promedios)
#podemos convertir a numpy array para realizar operaciones con
#variables de este tipo
promedios=promedios.to_numpy()
print(promedios)
#el modelo base de ref quedaría así:
print('\n El modelo base de referencia es mean_ren + mean_col - mean_tot:') 
#una media total (mean_tot):')
promedios[:,0]+promedios[:,1]-promedios[:,2]

,row,col,mean_score_by_row,mean_score_by_col,total_score_mean
row,,,,,
0,0,2,4.500000,3.333333,3.0
0,0,3,4.500000,3.000000,3.0
1,1,2,2.500000,3.333333,3.0
1,1,4,2.500000,3.250000,3.0
2,2,2,4.500000,3.333333,3.0
2,2,3,4.500000,3.000000,3.0
4,4,0,2.333333,1.000000,3.0
4,4,2,2.333333,3.333333,3.0
4,4,3,2.333333,3.000000,3.0


[[  0.      2.      4.5     3.333   3.   ]
 [  0.      3.      4.5     3.      3.   ]
 [  1.      2.      2.5     3.333   3.   ]
 [  1.      4.      2.5     3.25    3.   ]
 [  2.      2.      4.5     3.333   3.   ]
 [  2.      3.      4.5     3.      3.   ]
 [  4.      0.      2.333   1.      3.   ]
 [  4.      2.      2.333   3.333   3.   ]
 [  4.      3.      2.333   3.      3.   ]
 [  6.      2.      5.      3.333   3.   ]
 [  8.      3.      3.      3.      3.   ]
 [  9.      2.      1.      3.333   3.   ]
 [ 10.      1.      2.5     2.5     3.   ]
 [ 10.      3.      2.5     3.      3.   ]
 [ 12.      1.      3.5     2.5     3.   ]
 [ 12.      4.      3.5     3.25    3.   ]
 [ 13.      2.      3.      3.333   3.   ]
 [ 15.      1.      2.      2.5     3.   ]
 [ 15.      2.      2.      3.333   3.   ]
 [ 16.      1.      3.      2.5     3.   ]
 [ 16.      2.      3.      3.333   3.   ]
 [ 16.      3.      3.      3.      3.   ]
 [ 16.      4.      3.      3.25    3.   ]
 [ 17.     

array([-2.5  , -1.5  ,  0.5  ,  2.5  , -0.5  ,  0.5  ,  1.667,  3.667,
        4.667,  3.   ,  8.   , 10.   ,  8.5  , 10.5  ,  9.5  , 12.5  ,
       12.   , 14.   , 15.   , 14.   , 15.   , 16.   , 17.   , 17.   ,
       20.   ])

In [81]:
#Usuarios que calificaron más de 2 películas
display(filtrar(MC,'row',2))
display(filtrar(MC,'col',2))

array([[ 4,  2,  3],
       [ 4,  3,  3],
       [ 4,  0,  1],
       [16,  3,  1],
       [16,  1,  2],
       [16,  2,  4],
       [16,  4,  5]])

array([[16,  1,  2],
       [12,  1,  4],
       [10,  1,  3],
       [15,  1,  1],
       [15,  2,  3],
       [ 0,  2,  5],
       [ 1,  2,  2],
       [ 6,  2,  5],
       [ 4,  2,  3],
       [13,  2,  3],
       [16,  2,  4],
       [ 2,  2,  4],
       [ 9,  2,  1],
       [16,  3,  1],
       [ 2,  3,  5],
       [17,  3,  3],
       [ 4,  3,  3],
       [10,  3,  2],
       [ 8,  3,  3],
       [ 0,  3,  4],
       [18,  4,  2],
       [16,  4,  5],
       [ 1,  4,  3],
       [12,  4,  3]])

In [34]:
mf_engine=mf.MF()
type(mf_engine)

libmf.mf.MF

In [35]:
#fit method
mf_engine.fit(MC)
mf_engine

In [36]:
#testing factor-p
P=mf_engine.p_factors()
print('Matriz p:\n',P)
print('Tamaño p:\n',P.shape)

Matriz p:
 [[ 4.177e-01 -0.000e+00  2.319e-01  7.487e-02  2.519e-01  4.811e-01
   4.126e-01  6.502e-01]
 [ 6.727e-02  1.067e+00  6.691e-01  3.153e-01  4.735e-01 -6.606e-04
   3.509e-01  2.547e-01]
 [ 4.629e-01  6.447e-01  7.526e-01  7.045e-01  2.393e-01  2.458e-01
   4.882e-01  2.082e-01]
 [-2.677e-01  8.860e-01  1.133e+00  4.991e-01  9.346e-01  1.978e-01
   1.115e+00  4.990e-01]
 [ 7.422e-01  5.249e-01  3.339e-01  9.898e-02  7.368e-01  1.622e-01
   4.002e-01  3.446e-01]
 [ 7.761e-01  5.916e-02  8.939e-02  8.674e-01  7.410e-01  9.414e-01
   9.457e-01  1.058e+00]
 [ 2.031e+00  1.296e+00  2.948e-01  8.139e-01  5.543e-01  9.801e-01
   1.003e+00  1.800e+00]
 [ 7.833e-01  1.402e+00  9.135e-01  5.687e-01  1.199e+00  4.755e-01
   7.373e-01  9.758e-01]
 [       nan        nan        nan        nan        nan        nan
         nan        nan]
 [ 6.410e-01  5.901e-01  1.191e+00  6.070e-01  1.230e+00  1.125e+00
   1.211e+00  8.922e-01]
 [       nan        nan        nan        nan        nan   

In [37]:
#testing factor-q
Q=mf_engine.q_factors()
print('Matriz Q:\n',Q)
print('Tamaño Q:\n',Q.shape)

Matriz Q:
 [[ 0.495  1.047  0.892  0.886  0.44   0.263  0.897  0.355]
 [ 1.9    0.838 -0.027  0.626  0.221  0.817  0.597  1.511]
 [ 0.485 -0.191 -0.033  0.608  0.308  0.852  0.69   0.862]
 [ 0.623  0.119  0.     0.529 -0.064  0.045 -0.063  0.068]
 [ 0.534  1.35   1.001  0.647  0.881  0.741  1.06   0.798]
 [ 0.202  0.58   0.987  0.594  0.74   0.442  1.131  1.015]
 [ 0.723  0.332  0.585  0.896  0.303  0.88   0.639  0.618]
 [   nan    nan    nan    nan    nan    nan    nan    nan]
 [-0.083  0.056 -0.039  0.226 -0.113  0.407  0.505  0.065]
 [ 0.505  0.385  0.712  0.615  0.712  0.845  0.556  0.19 ]
 [   nan    nan    nan    nan    nan    nan    nan    nan]
 [   nan    nan    nan    nan    nan    nan    nan    nan]
 [ 0.734  0.784  1.22   0.651  1.33   1.119  1.105  1.019]
 [ 0.847  1.773  1.281  0.619  1.45   0.876  1.262  1.245]
 [   nan    nan    nan    nan    nan    nan    nan    nan]
 [   nan    nan    nan    nan    nan    nan    nan    nan]
 [   nan    nan    nan    nan    nan    nan  

In [38]:
#np.where(np.isnan(P), 0, P)@np.transpose(np.where(np.isnan(Q), 0, Q))
R_est=P@np.transpose(Q)
print('Matriz p:\n',R_est)
print('Tamaño p:\n',R_est.shape)

Matriz p:
 [[  1.318   2.512   1.574   0.324   2.038   1.883   1.67      nan   0.391
    1.362     nan     nan   2.63    2.814     nan     nan     nan     nan
      nan   3.062]
 [  2.64    1.9     0.606   0.3     3.342   2.486   1.602     nan   0.239
    1.696     nan     nan   3.184   4.448     nan     nan     nan     nan
      nan   3.279]
 [  2.881   2.7     1.305   0.717   3.403   2.678   2.35      nan   0.461
    2.14      nan     nan   3.567   4.373     nan     nan     nan     nan
      nan   4.399]
 [  3.887   2.303   1.623   0.116   5.059   4.42    2.689     nan   0.711
    2.867     nan     nan   5.41    6.661     nan     nan     nan     nan
      nan   5.457]
 [  2.151   2.958   1.247   0.535   2.972   2.262   1.83      nan   0.185
    1.826     nan     nan   3.383   4.193     nan     nan     nan     nan
      nan   3.422]
 [  3.092   5.16    3.485   0.957   4.342   3.902   3.722     nan   0.977
    3.063     nan     nan   5.451   5.823     nan     nan     nan     nan
      

In [39]:
#validación cruzada
mf_engine.mf_cross_validation(MC)

3.200094887204332

In [82]:
pelis_nombres=pd.read_csv('movies_title_fix.csv',names=["peli_id", "año", "nombre"])

In [83]:
pelis_nombres.head(10) #para ver cómo se encuentran los datos

,peli_id,año,nombre
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW
5,6,1997.0,Sick
6,7,1992.0,8 Man
7,8,2004.0,What the #$*! Do We Know!?
8,9,1991.0,Class of Nuke 'Em High 2
9,10,2001.0,Fighter


In [85]:
df_muestra_netflix=pd.read_csv('dat_muestra_nflix.csv')

In [86]:
df_muestra_netflix.head

<bound method NDFrame.head of           peli_id  usuario_id_orig  calif       fecha  usuario_id
0               1             2442      3  2004-04-14           1
1               1          1086807      3  2004-12-28           2
2               1          2165002      4  2004-04-06           3
3               1          1133214      4  2004-03-07           4
4               1          1537427      4  2004-03-29           5
...           ...              ...    ...         ...         ...
20968936    17770           635735      4  2004-12-01       32710
20968937    17770          1983962      2  2005-02-23       60537
20968938    17770          1274035      4  2005-06-10        3921
20968939    17770           516110      5  2004-08-26        5967
20968940    17770           311124      3  2005-09-29        4674

[20968941 rows x 5 columns]>

In [ ]:
# #testing predict method
# pred_data = mf.generate_test_data(9, 4, 5, indices_only=True)
# predictions = mf_engine.predict(pred_data)


In [ ]:
# #$ python tests/mf_tests.py
# %run 'mf_tests.py'


In [27]:
# import csv
# with open('dat_muestra_nflix.csv', newline='') as csvfile:
#     spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')

In [1]:
# spamreader[1]

In [ ]:
# data = pd.read_csv("dat_muestra_nflix.csv", sep=' ')

In [ ]:
# def centralizar_valores(M):
#     n=M.shape[0]
    
 
    
#     #Se hace una copia de la matriz M
#         M_copy=copy.copy(M)
#         #Se agrega una 4ta columna
#         M_copy=np.append(M, np.array(n*[np.nan]).reshape(n,1), axis=1)
#         #Se ordena la matriz
#         M_copy=M_copy[np.argsort(M_copy[:,0])]
#         #k1 irá marcando el renglón donde van cambiando los valores de la 1era columna de M
        
        
#         valores=np.zeros(1)
#         for i in range(1,M_copy.shape[0]):
#             if M_copy[i,2]!=M_copy[i-1,2]:
#                 print('lo que sea')
            
            
#             valores[k1]=np.r_[valores, np.zeros(1)]
#             for j in range(k1,k2+1):
#                 #M_copy[j,4]
#                 print('lo que sea')
# #     else:
# #         calif_cent=np.nanmean(M, axis=1).reshape(n,1)
# #         return M-calif_cent

In [ ]:
# help(mf)

In [ ]:
# print([getattr(mf, a)
# for a in dir(mf)
#   if isinstance(getattr(mf, a), types.FunctionType)])

In [ ]:
#help(mf.generate_test_data)

In [ ]:
#dir(mf_engine)

In [ ]:
# #Uso de función generate_test_data
# #para generar un dataset de prueba con mil renglones, 
# #cuyos elementos de la 1era, columna van de 0 a 100, 
# #y los de la segunda columna de 0 a 50, la tercera 
# #columna son numeros aleatorios 
# test_data = mf.generate_test_data(9, 4, 20)
# print('Matriz de prueba:\n',test_data)
# print('Tamaño de la matriz:\n',test_data.shape)

In [ ]:
# def myFun(arg1, *argv): 
#     print(type(argv))
#     if len(argv)==0:
#         print('no hay argumentos variables')
#     print ("First argument :", arg1) 
#     for arg in argv: 
#         print("Next argument through *argv :", arg)
# #myFun(['hola','tu','chales'])
# myFun(*[3])

In [ ]:
# nan_or_value_vec_one_arg=np.vectorize(nan_or_value)
# nan_or_value_vec_mult_args=np.vectorize(nan_or_value, otypes=[object])
# def nan_or_value_vec(M,f,*args):
#     if len(args)==1:
#         print('usase la función constante?')
#         return nan_or_value_vec_one_arg(M,f,args)
#     else:
#         return nan_or_value_vec_mult_args(M,f,*args)

In [1009]:
# print('Numero de elementos distintos de cero:\n',np.count_nonzero(R))

In [ ]:
#display(pd.DataFrame(MC).sort_values(by=['row','col'], ascending=True))